In [4]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np

import geopandas as gpd
import seaborn as sns

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

import shapely
from shapely.geometry import LineString, MultiPoint
from shapely.ops import split, substring

import zlib

import datetime as dt

### get total peak headways!!

* since specified as "and", get both periods and take the min...

In [5]:
##check this...
am_peak = (6, 9)
pm_peak = (3, 7)

In [6]:
am_commute_hours = list(am_peak)
pm_commute_hours = list(pm_peak)
commute_hours = am_commute_hours + pm_commute_hours

#### Flow

* if next shape contains existing segment, ignore and clip
* calculate new segments on non-overlap (which may be discontiguous?)

In [7]:
def split_every_km(geometry):
    lines = []
    
    geometry = geometry.iloc[0]
    if hasattr(geometry, 'geoms'): ##check if MultiLineString
        linestrings = geometry.geoms
    else:
        linestrings = [geometry]
    for linestring in linestrings:
        for i in range(0, int(linestring.length), 1000):
    #         print(i)
            lines.append(substring(linestring, i, i+1000))
    return lines

In [9]:
def find_stop_with_most_routes(segment, stops, route_count_by_stop):
    stops_in_seg = gpd.clip(stops, segment.geometry) 
#     print(f'stops_in_seg {stops_in_seg}')
    max_routes_stop = (stops_in_seg
                   >> inner_join(_, route_count_by_stop, on = "stop_id")
                   >> filter(_.n_routes == _.n_routes.max())
                  ).iloc[[0]]
#     display(max_routes_stop[['stop_id', 'n_routes']])
#     print(f'max routes stop: {max_routes_stop}')
    segment['stop_id'] = max_routes_stop['stop_id'].iloc[0]
    segment['n_routes'] = max_routes_stop['n_routes'].iloc[0]
    return segment

In [10]:
def get_operator_views(itp_id):
        shapes = tbl.gtfs_schedule.shapes() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        shapes = gpd.GeoDataFrame(shapes, 
                                  geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
                                  crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
        trips = tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stop_times = tbl.gtfs_schedule.stop_times() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stops = (tbl.gtfs_schedule.stops() 
                 >> filter(_.calitp_itp_id == itp_id)
                 >> select(_.stop_id, _.stop_lat, _.stop_lon)
                 >> collect())
        
        return shapes, trips, stop_times, stops

In [27]:
def fix_arrival_time(gtfs_timestring):
    split = gtfs_timestring.split(':')
    hour = int(split[0])
    if hour >= 24:
#         print('fixing')
#         print((':').join(split))
        split[0] = str(hour - 24)
        corrected = (':').join(split)
#         print(corrected)
        return corrected
    else:
#         print('no_fix')
        return gtfs_timestring

In [29]:
def single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta):
    global _debug
    global _debug2, _debug3, _debug4
    
    print(shape_id)
    
    single_shape = (shapes
         >> filter(_.shape_id == shape_id)
         >> mutate(shape_pt_sequence = _.shape_pt_sequence.astype('int64'))
         >> arrange(_.shape_pt_sequence) ##arrange, then convert to line to preserve order...
        )
    
    route_line = LineString(list(single_shape['geometry']))
    single_line = single_shape[['calitp_itp_id', 'shape_id', 'calitp_extracted_at']].iloc[[0]] ##preserve info cols
    single_line['geometry'] = route_line
    single_line = gpd.GeoDataFrame(single_line, crs='EPSG:6414')
        ## test a shapes df split into segments
    _debug = single_line
    if hqta.size != 0:
#         display(hqta.head(2))
#         display(single_line.head(2)
        already_calculated = hqta.dissolve(by='calitp_itp_id')
        single_line = single_line.overlay(already_calculated, how='difference') ##doesn't like this?
        if single_line.size == 0:
            print('no line')
            segments_with_max_stop = None
            return
#     else:
#         print('else!')
    segmented = pd.DataFrame()
    _debug2 = single_line
    for segment in split_every_km(single_line.geometry): ##working
        to_append = single_line.drop(columns=['geometry'])
        to_append['geometry'] = segment
#         to_append['segment_sequence'] = segment[1]
    #     print(to_append)
        segmented = segmented.append(to_append)
        
    segmented = segmented.reset_index()
    segmented['segment_sequence'] = segmented.index.astype(str)
    
    segmented = segmented.astype({'calitp_itp_id': str})
    ## compute (hopefully unique) hash of segment id that can be used across routes/operators
#     _debug2 = segmented
    segmented['hqta_segment_id'] = segmented.apply(lambda x:
                        zlib.crc32((x.calitp_itp_id + x.shape_id + x.segment_sequence).encode('utf-8')),
                                                   axis=1)
    segmented.geometry = segmented.buffer(50) ##generous buffer for street/sidewalk width?
    
    try:
        _debug = segmented
        segments_with_max_stop = segmented.apply(find_stop_with_most_routes, axis=1,
                                                 args = (stops, route_count_by_stop))
    except IndexError:
        print('no stops')
        segments_with_max_stop = None
        return ##no stops in segment

    max_stop_times = (stop_times 
                 >> select(_.stop_id, _.trip_id, _.departure_time)
                 >> inner_join(_, segments_with_max_stop, on = "stop_id")
                )

        ##only works for whole hours, but should be fine?
    _debug3 = max_stop_times
    
    ##TODO debug/simplify
    max_stop_times['departure_time'] = max_stop_times['departure_time'].apply(fix_arrival_time)
    max_stop_times['departure_hour'] = max_stop_times['departure_time'].apply(lambda x:
                                                                    dt.datetime.strptime(x, '%H:%M:%S').hour)
    max_stop_times = max_stop_times >> filter(_.departure_hour.isin(commute_hours))
#     max_stop_times = max_stop_times[max_stop_times['departure_hour'].isin(commute_hours)]
    max_stop_times['am_peak'] = max_stop_times['departure_hour'].apply(lambda x: x in am_commute_hours)
    max_stop_times['pm_peak'] = ~max_stop_times['am_peak']
    _debug4 = max_stop_times
    segment_peak_service = max_stop_times.groupby(['hqta_segment_id'])[['am_peak', 'pm_peak']].sum()
    segment_peak_service['am_peak'] = (segment_peak_service['am_peak'] / len(am_commute_hours)).round(1)
    segment_peak_service['pm_peak'] = (segment_peak_service['pm_peak'] / len(pm_commute_hours)).round(1)

    segment_peak_service['hq_transit_corr'] = segment_peak_service.apply(lambda x:
                              True if x.am_peak > 4 and x.pm_peak > 4 else False, axis=1)

    segment_peak_service = segment_peak_service.reset_index() ## All segments hq_corridors, which makes sense for BBB1

    single_hqta = segments_with_max_stop >> inner_join(_, segment_peak_service, on = 'hqta_segment_id')
    single_hqta.drop(columns=['calitp_extracted_at'], inplace=True)

    print(f'appending {single_hqta.shape}')
#     display(single_hqta)
    return single_hqta

In [30]:
def single_operator_hqta(views, hqta_df=None):
    global _result
    
    shapes, trips, stop_times, stops = views
    
    stops = gpd.GeoDataFrame(stops,
                             geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                             crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    distinct_routes = (trips
                   >> distinct(_.route_id, _.shape_id, _.direction_id, _keep_all = True)
                   >> select(_.calitp_itp_id, _.route_id, _.shape_id, _.direction_id, _.trip_id))
    route_count_by_stop = (stop_times
                         >> select(_.stop_id, _.trip_id)
                         >> inner_join(_, distinct_routes, on = "trip_id")
                         >> count(_.stop_id)
                         >> rename(n_routes = _.n)
                         >> arrange(-_.n_routes))

    hqta = gpd.GeoDataFrame()
    for shape_id in views[0]['shape_id'].unique():
        print(shape_id)
#         display(hqta.tail(2))
        print('...')
        try:
            _result = single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta)
            hqta = hqta.append(_result)
        except:
            print(f'unable to calculate HQTA for shape_id {shape_id}')
        try:
            hqta = hqta.set_crs('EPSG:6414')
        except:
            continue
        
    return hqta

In [14]:
# bbb_test_shape_ids = bbb_views[0]['shape_id'].unique()
# # bbb_test_shape_ids = ['25386', '25387', '25311', '25313', '25314', '25315', '25316']
# # bbb_test_shape_ids = ['25311', '25313']
# # bbb_test_shape_ids = ['25368']
# # bbb_test_shape_ids = ['25327', '25328', '25329']
# # bbb_test_shape_ids = ['25327']

In [15]:
## passed for single op + single shape!
# bigtest = single_operator_hqta(300)

In [16]:
# bbb_views = get_operator_views(300)

In [17]:
# bt2 = single_operator_hqta(bbb_views)

In [18]:
act_views = get_operator_views(200)

In [ ]:
act_hqta = single_operator_hqta(act_views)

RV:p_695
...
RV:p_695
no stops
CE:r9ei
...
CE:r9ei
no stops
CE:w59p
...
CE:w59p
no stops
SA:p_897679
...
SA:p_897679
no stops
SA:p_897680
...
SA:p_897680
no stops
DE:DB0083
...
DE:DB0083
no stops
DE:DB10036
...
DE:DB10036
no stops
DE:Dumbarton_Express_Consortium27:DB0084
...
DE:Dumbarton_Express_Consortium27:DB0084
no stops
DE:Dumbarton_Express_Consortium27:DB10036
...
DE:Dumbarton_Express_Consortium27:DB10036
no stops
DE:Dumbarton_Express_Consortium27:DB10038
...
DE:Dumbarton_Express_Consortium27:DB10038
no stops
DE:Dumbarton_Express_Consortium27:DB10039
...
DE:Dumbarton_Express_Consortium27:DB10039
no stops
DE:Dumbarton_Express_Consortium33:DB0084
...
DE:Dumbarton_Express_Consortium33:DB0084
no stops
DE:Dumbarton_Express_Consortium33:DB10036
...
DE:Dumbarton_Express_Consortium33:DB10036
no stops
DE:Dumbarton_Express_Consortium33:DB10037
...
DE:Dumbarton_Express_Consortium33:DB10037
no stops
DE:Dumbarton_Express_Consortium33:DB10038
...
DE:Dumbarton_Express_Consortium33:DB10038
no sto

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (13, 11)
AC:shp-10-10
...
AC:shp-10-10


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (2, 11)
AC:shp-12-13
...
AC:shp-12-13
no line
AC:shp-12-56
...
AC:shp-12-56
no line
AC:shp-18-16
...
AC:shp-18-16
no line
AC:shp-18-17
...
AC:shp-18-17
no line
AC:shp-19-64
...
AC:shp-19-64
no line
AC:shp-1T-05
...
AC:shp-1T-05


...

appending (2, 11)
PE:p_110930
...
PE:p_110930
no line
PE:p_177274
...
PE:p_177274
no line
PE:p_177275
...
PE:p_177275
no line
PE:p_4743
...
PE:p_4743
no line
CT:p_1277284
...
CT:p_1277284
no stops
CT:p_1277345
...
CT:p_1277345
no stops
CT:p_1277353
...
CT:p_1277353
no stops
CT:p_1277405
...
CT:p_1277405
no stops
FS:p_1276663
...
FS:p_1276663
no stops
FS:p_1276696
...
FS:p_1276696
no stops
FS:p_1276817
...
FS:p_1276817
no stops
FS:p_2689
...
FS:p_2689
no line
FS:p_900130
...
FS:p_900130
no line
FS:p_915149
...
FS:p_915149
no line
WC:p_5168
...
WC:p_5168
no stops
WC:p_5197
...
WC:p_5197
no stops
WC:p_5199
...
WC:p_5199
no stops
WC:p_5216
...
WC:p_5216
no stops
WC:p_5227
...
WC:p_5227
unable to calculate HQTA for shape_id WC:p_5227
WC:p_786724
...
WC:p_786724
unable to calculate HQTA for shape_id WC:p_786724
WC:p_786725
...
WC:p_786725
unable to calculate HQTA for shape_id WC:p_786725
WC:p_786739
...
WC:p_786739
unable to calculate HQTA for shape_id WC:p_786739
MA:10
...
MA:10
no stops
MA

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (11, 11)
MA:128
...
MA:128
no stops
MA:16
...
MA:16
no stops
MA:178
...
MA:178
no line
MA:25
...
MA:25
no stops
MA:36
...
MA:36
no stops
MA:49
...
MA:49
no stops
MA:53
...
MA:53
no stops
MA:65
...
MA:65
no stops
MA:72
...
MA:72
no stops
MA:80
...
MA:80
no stops
3D:27
...
3D:27
no stops
3D:43
...
3D:43
no stops
3D:46
...
3D:46
no stops
3D:48
...
3D:48
no stops
3D:50
...
3D:50
no stops
VC:p_178725
...
VC:p_178725
no line
SM:1100256
...
SM:1100256


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (15, 11)
SM:1210347
...
SM:1210347
no stops
SM:1210348
...
SM:1210348
no stops
SM:1300277
...
SM:1300277
no stops
SM:1410284
...
SM:1410284
no stops
SM:170514
...
SM:170514
no stops
SM:170520
...
SM:170520


#### Flow

* segment table should be global for all operators...
* if shape contains existing segments, add them
* calculate new segments on non-overlap (which may be discontiguous?)
* should yield an operator+segment table that can be appended+grouped+summed with other operators...

### Mapping

In [ ]:
def map_hqta(hqta):
    
    x = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.x
    y = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)
    geo_data_hq = GeoData(geo_dataframe = hqta[hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#3366cc',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'HQTA')

    geo_data_not_hq = GeoData(geo_dataframe = hqta[~hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#fec44f',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'non-HQTA')

    m.add_layer(geo_data_hq)
    m.add_layer(geo_data_not_hq)
    m.add_control(LayersControl())
    
    return m

In [ ]:
map_hqta(act_hqta)

In [ ]:
def simple_map(gdf, mouseover=None):
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
        gdf.geometry = gdf.geometry.buffer(50)
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[34, -118.34], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
    
    geo_data_hq = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#3366cc',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'gdf')
    m.add_layer(geo_data_hq)
    
    if mouseover:
        geo_data_hq.on_hover(update_html)

    return m